In [1]:
from surface_dynamics.all import Origami
import numpy as np
from itertools import permutations 

In [2]:
def decomposeHU(n):
    """
    Input an integer n, output a list of all possible decomposition of (h,u)
    such that h*u = n
    """
    output = []
    for i in range(int(np.sqrt(n))):
        j = i+1
        if (n/j) == (n//j):
            output.append(np.array((j,n/j)))
    return output

#decomposeHU(1)

In [3]:
def decomposeNN(n):
    """
    Input an integer n, output a list of all possible decomposition of (n1, n2)
    such that n1+n2 = n
    """
    output = []
    for i in range(n):
        j = i+1
        output.append(np.array((j,n-j)))
    return output

#decomposeNN(3)

In [4]:
def decomposeHUHU(n):
    """
    Input an integer n, output a list of all possible decompositions of
    (h1, u1, h2, u2) such that h1*u1+h2*u2 = n, and u1 < u2
    """
    output = []
    n1_n2 = decomposeNN(n)
    for n1, n2 in n1_n2:
        h1_u1 = decomposeHU(n1)
        h2_u2 = decomposeHU(n2)
        for h1, u1 in h1_u1:
            for h2, u2 in h2_u2:
                if (u1 < u2):
                    new = np.array((h1,u1,h2,u2)).astype(int)
                    output.append(new)
    return output

#decomposeHUHU(4)

In [5]:
def up_right_pairs(h,u):
    """
    Input a rectangle with height h and width w, construct up and right permutation pieces
    """
    up_pairs = []
    right_pairs = []
    for i in range(u):
        up_pairs.append(np.arange(i+1+(h-1)*u,i,-u))
    for i in range(h):
        right_pairs.append(np.arange(1+i*u,1+(i+1)*u,1))
    return up_pairs, right_pairs

In [6]:
def two_up_right_pairs(h1,u1,h2,u2):
    """
    Input a rectangle with height h and width w, construct up and right permutation pieces
    """
    up_pairs = []
    right_pairs = []
    
    for i in range(u1):
        half_up = np.arange(i+1+(h1-1)*u1,i,-u1)
        half_down = np.arange(h1*u1+i+1+(h2-1)*u2,h1*u1+i,-u2)
        one = np.concatenate((half_down, half_up))
        up_pairs.append(one)
    for i in range(u1,u2):
        up_pairs.append(np.arange(h1*u1+i+1+(h2-1)*u2,h1*u1+i,-u2))
    for i in range(h1):
        right_pairs.append(np.arange(1+i*u1,1+(i+1)*u1,1))
    for i in range(h1,h1+h2):
        right_pairs.append(np.arange(1+h1*u1+(i-h1)*u2,1+h1*u1+(i+1-h1)*u2,1))
    return up_pairs, right_pairs

#up_pairs, right_pairs = two_up_right_pairs(3,5,2,8)
#print(right_pairs)

In [7]:
def arr_to_circ_per(arr,n):
    """
    Input an array or tuple representing a permutation in the array form, 
    output the same permutation using circular notation, as a list of arrays
    """
    circ = []
    index = []
    for j in range(n):
        if (j not in index):
            #length = 0
            piece = []
            piece.append(j)
            index.append(j)
            head = j
            tail = arr[j]
            while (tail != head):
                #length = length + 1
                piece.append(tail)
                index.append(tail)
                tail = arr[tail]
            piece = np.array(piece)
            circ.append(piece)
    return circ

#arr_to_circ_per((2,1,0),3)

In [8]:
def circ_per(n):
    """
    Input a n, output a list of all the permutation using circular notation,
    each circular permutation is represented as a list of arrays
    """
    l = list(permutations(range(0, n)))
    for i in range(len(l)):
        inter = l[i]
        circ = arr_to_circ_per(inter,n)
        l[i] = circ
    return l

#circ_per(3)

In [9]:
def genarate_origami_per(pairs,index_circ_per):
    """
    Input a list of up/right arrays of origamis, and the permutation of index to glue them,
    which is represented using the circular notation, output the glued permutation of the 
    origamis as a list of arrays
    
    index_circ_per contains all the circles using a list of arrays, i.e [(0,1,2),(3,5,4)]
    """
    list_per = []
    for circle in index_circ_per:
        list_circ = []
        for i in circle:
            for j in pairs[i]:
                list_circ.append(j)
        circ = np.array(list_circ)
        circ = circ.reshape(-1,)
        list_per.append(circ)
    return list_per

#all_up_circ = circ_per(8)
#index_circ_per = all_up_circ[3]
#up_pairs, right_pairs = two_up_right_pairs(3,5,2,8)

In [10]:
def genarate_all_per(pairs,n):
    """
    Input a list of n permutation pieces, give all possible permutations 
    by gluning the pieces together
    """
    all_circ = circ_per(n)
    list_per = []
    for circ in all_circ:
        pair_circ = genarate_origami_per(pairs,circ)
        list_per.append(pair_circ)
    return list_per

up_pairs, right_pairs = two_up_right_pairs(2,3,2,4)
up_list_per = genarate_all_per(up_pairs,4)
#rigth_list_per = genarate_all_per(right_pairs,3)

In [11]:
def list_to_string_per(list_per):
    """
    Take in a circular permutation represented as a list of arrays,
    output the circular permutation represented as a string
    """
    outstr = ''
    for arr in list_per:
        arr2 = np.array((arr, arr)).reshape(-1,)
        l = arr2.shape[0]
        if (l != 2):
            l = arr.shape[0]
            onestr = '('
            for i in range(l-1):
                onestr = onestr + f'{arr[i]}, '
            onestr = onestr + f'{arr[l-1]})'
            outstr = outstr + onestr
    return outstr

#list_per = [np.array((1,2)),np.array((0))]
#list_to_string_per(list_per)

In [12]:
def permutation_rectangle(h,u):
    """
    Input a rectangle with height h and width w, output all possible right and up permutations
    to construct a rectangle surface
    """
    up_pairs,right_pairs = up_right_pairs(h,u)
    all_up_per = genarate_all_per(up_pairs,u)
    all_right_per = genarate_all_per(right_pairs,h)
    pair_per = []
    for up_per in all_up_per:
        for right_per in all_right_per:
            up = list_to_string_per(up_per)
            right = list_to_string_per(right_per)
            pair_per.append((up,right))
    return pair_per
            

#permutation_rectangle(3,2)

In [13]:
def origami_rectangle(h,u):
    """
    Input a rectangle with height h and width w, output all unique origamis
    """
    pair_per = permutation_rectangle(h,u)
    Sx = []
    for pair in pair_per:
        up = pair[0]
        right = pair[1]
        origami = Origami(up, right)
        flag = 0
        for oi in Sx:
            if (origami.is_isomorphic(oi)):
                flag = 1
                break
        if (flag == 0):
            Sx.append(origami)
    return Sx

#S = origami_rectangle(3,2)
#S[2].plot()

In [14]:
def two_permutation_rectangle(h1,u1,h2,u2):
    """
    Input two rectangles with height h1, h2 and width u1, u2, output all possible right and up permutations
    to construct a surface
    """
    up_pairs,right_pairs = two_up_right_pairs(h1,u1,h2,u2)
    all_up_per = genarate_all_per(up_pairs,u2)
    all_right_per = genarate_all_per(right_pairs,h1+h2)
    pair_per = []
    for up_per in all_up_per:
        for right_per in all_right_per:
            up = list_to_string_per(up_per)
            right = list_to_string_per(right_per)
            pair_per.append((up,right))
    return pair_per

In [15]:
def two_origami_rectangle(h1,u1,h2,u2):
    """
    Input two rectangles with height h1, h2 and width u1, u2, output all unique origamis
    """
    pair_per = two_permutation_rectangle(h1,u1,h2,u2)
    Sx = []
    for pair in pair_per:
        up = pair[0]
        right = pair[1]
        origami = Origami(up, right)
        #Sx.append(origami)
        flag = 0
        for oi in Sx:
            if (origami.is_isomorphic(oi)):
                flag = 1
                break
        if (flag == 0):
            Sx.append(origami)
    #print(len(Sx))
    return Sx

#S = two_origami_rectangle(2,3,2,4)
#S[2].plot()

In [16]:
def construct_Sn(n):
    """
    Input an integer n, output a list of all square-tilled surfaces in Sn
    """
    Sn = []
    hhuu_pairs = decomposeHUHU(n)
    for h1,u1,h2,u2 in hhuu_pairs:
        Si = two_origami_rectangle(h1,u1,h2,u2)
        Sn = Sn + Si
    print(len(Sn))
    return Sn

Sn = construct_Sn(6)

244


In [17]:
def double_check_construct_Sn(n):
    """
    Input an integer n, output a list of all square-tilled surfaces in Sn
    """
    Sn = []
    hhuu_pairs = decomposeHUHU(n)
    for h1,u1,h2,u2 in hhuu_pairs:
        Si = two_origami_rectangle(h1,u1,h2,u2)
        for oi in Si:
            flag = 0
            for on in Sn:
                if (oi.is_isomorphic(on)):
                    flag = 1
                    break
            if (flag == 0):
                Sn.append(oi)
    print(len(Sn))
    return Sn

Sn = double_check_construct_Sn(6)

234


In [18]:
Sn = construct_Sn(7)

1501


In [20]:
Sn = double_check_construct_Sn(7) 
#result shows that there's still posibility for the first method to have isomorphic origamis, but not much

1399
